Transformed Sectiontion Method
- "Its core principle involves conceptually converting the multi-material cross-section of a layered beam into an equivalent, fictitious cross-section composed entirely of a single, homogeneous material."
- "The transformation is achieved by adjusting the width of each material by a factor equal to its modular ratio (n_i = E_i / E_ref), relative to the chosen base material."

(hand hardness, grain form) -> density
density -> Youngs Modulus
(Young's Modulus, thickness, orientation) -> Bending stiffness

In [6]:
# Import Libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from snowpylot import caaml_parser


## Outline Transformed Section Method

**Layer Properties**

    rho_i: density of layer i, kg/m^3
    b_i=1: width of layer i, m (start with 1m as baseline)
    h_i: # thickness of layer i

**Calculation**

1. Calculate density from hand hardness and grain form

2. Calculate Young's Modulus from density

    **rho_0** = 917 # kg/m^3 (density of ice), from "A Closed form model..."

    **E_i**= 6.5e3 *(rho_i)**4.4 # From Gerling et al. (2017)

3. Select Reference Material in Slab (highest E)

    **E_ref** = max(E_i) # Choose material with highest E

4. Calculate modular ratio for each layer

    **n_i** = E_i / E_ref

5. Calculate transformed width of each layer

    **b_i_prime** = b_i * n_i # m (transformed width of layer i)

6. Calculate transformed area of each layer

    **A_i_prime** = b_i_prime * h_i # m^2 (transformed area of layer i)

7.  Locate Centroid of Layer

    **y_i** = sum of thickness of layers below i + h_i/2

8.  Locate Centroid of Transformed slab

    **Y_bar** = sum(A_i_prime * y_i) / sum(A_i_prime) # m (centroid of transformed slab)

9.  Calculate Moment of Inertia (I) of the Transformed Slab

    **I_i** = (1/12) * b_i_prime * h_i**3 # m^4 (moment of inertia of layer i)

    **d_i** = abs(y_i-Y_bar) # m (distance from centroid of layer i to centroid of transformed slab)

    **I_bar** = sum(I_i + A_i_prime * d_i**2) # m^4 (moment of inertia of transformed slab) NOTE: CHECK THIS EQUATION

10. Calculate Equivalent Bending Stiffness (D11) of the Transformed Slab

    **D11** = E_ref * I_bar # N*m^2 (bending stiffness of transformed slab)


parse_pits function: Parses all pits in a specified folder

In [7]:
def parse_pits(folder_path):
    """
    Function to parse CAAML files in the specified folder
    """

    files = [
        f for f in os.listdir(folder_path) if f.endswith(".xml")
    ]  # List of all .xml files in the folder

    pits_list = []

    for file in files:  # iterate through each file in the folder
        file_path = folder_path + "/" + file  # create the file path
        pit = caaml_parser(file_path)  # parse the file
        pits_list.append(pit)

    return pits_list


Specify folders for 2020-2024 Water Years and parse files

In [8]:
# Define folders and parse pits

pits_19_20 = parse_pits("../snowpits/by_season/2019-2020")
pits_20_21 = parse_pits("../snowpits/by_season/2020-2021")
pits_21_22 = parse_pits("../snowpits/by_season/2021-2022")
pits_22_23 = parse_pits("../snowpits/by_season/2022-2023")
pits_23_24 = parse_pits("../snowpits/by_season/2023-2024")

all_pits = (
    pits_19_20 + pits_20_21 + pits_21_22 + pits_22_23 + pits_23_24
)  # list of all pits


Geldsetzer table of density from hand hardness and grain form

In [9]:
geldsetzer_df = pd.read_csv('geldsetzer_table.csv', index_col=0)

def get_density(hand_hardness, grain_form, df=geldsetzer_df):
    """
    Get density value for a specific hand hardness and grain form combination.
    
    Parameters:
    df (pd.DataFrame): The Geldsetzer table DataFrame
    hand_hardness (str): Hand hardness value (e.g., 'F-', '4F+', 'P-', etc.)
    grain_form (str): Grain form (e.g., 'PP', 'DF', 'RG', 'FC', 'DH', etc.)
    
    Returns:
    float: Density value, or NaN if not available
    """
    try:
        return df.loc[hand_hardness, grain_form]
    except KeyError:
        return np.nan

def convert_grain_form(layer):
    """
    Convert grain form to code needed for Geldsetzer table.
    
    Parameters:
    layer: Snow layer object with grain_form_primary attribute
    
    Returns:
    str: Grain form code for Geldsetzer table lookup
    """
    if layer.grain_form_primary.sub_grain_class_code in ["PPgp","RGmx","FCmx"]:
        return layer.grain_form_primary.sub_grain_class_code
    else:
        return layer.grain_form_primary.basic_grain_class_code


In [10]:
def calculate_weight_above_layer(layers, target_layer):
    """
    Calculate cumulative weight above a specified target layer (excludes target layer).

    Parameters:
    layers: List of snow layer objects
    target_layer: The layer to stop calculation at (weight calculated above this layer)
    
    Returns:
    float: Weight above target layer in kg/m², or NaN if calculation fails
    """
    weight_above = 0
    target_found = False
    
    for layer in layers:
        # Check if we've reached the target layer - if so, stop BEFORE including it
        if layer is target_layer:
            target_found = True
            break
            
        # Convert grain form to code needed for Geldsetzer table
        grain_form = convert_grain_form(layer)
        
        # Get density
        density = get_density(layer.hardness, grain_form)
        
        if pd.isna(density):
            return np.nan
            
        # Calculate weight
        thickness_m = (layer.thickness[0])/100  # convert cm to m
        weight = thickness_m * density  # Weight in kg (assuming 1 m^2 cross-section)
        weight_above += weight
    
    # If target layer was not found, return NaN
    if not target_found:
        return np.nan
        
    return weight_above


In [12]:
def calculate_bending_stiffness(layers, target_layer):
    """
    Calculate bending stiffness above a specified target layer using the Transformed Section Method.
    
    Parameters:
    layers: List of snow layer objects
    target_layer: The layer to stop calculation at (bending stiffness calculated for slab above this layer)
    
    Returns:
    float: Bending stiffness (D11) in N*m^2, or NaN if calculation fails
    """
    
    # Collect layer properties for layers above the target layer
    layer_properties = []
    target_found = False
    
    for layer in layers:
        # Check if we've reached the target layer - if so, stop BEFORE including it
        if layer is target_layer:
            target_found = True
            break
            
        # Convert grain form to code needed for Geldsetzer table
        grain_form = convert_grain_form(layer)
        
        # Get density
        density = get_density(layer.hardness, grain_form)
        
        if pd.isna(density):
            return np.nan
            
        # Calculate layer properties
        thickness_m = (layer.thickness[0])/100  # convert cm to m
        
        # Step 2: Calculate Young's Modulus from density
        E_i = 6.5e3 * (density**4.4)  # From Gerling et al. (2017)
        
        # Store layer properties
        layer_properties.append({
            'density': density,
            'thickness': thickness_m,
            'E_i': E_i,
            'b_i': 1.0  # width of layer i, m (baseline = 1m)
        })
    
    # If target layer was not found or no layers above it, return NaN
    if not target_found or len(layer_properties) == 0:
        return np.nan
    
    # Step 3: Select Reference Material in Slab (highest E)
    E_ref = max(layer_prop['E_i'] for layer_prop in layer_properties)
    
    # Steps 4-6: Calculate modular ratio, transformed width, and transformed area
    for layer_prop in layer_properties:
        layer_prop['n_i'] = layer_prop['E_i'] / E_ref  # modular ratio
        layer_prop['b_i_prime'] = layer_prop['b_i'] * layer_prop['n_i']  # transformed width
        layer_prop['A_i_prime'] = layer_prop['b_i_prime'] * layer_prop['thickness']  # transformed area
    
    # Step 7: Locate Centroid of each Layer
    for i, layer_prop in enumerate(layer_properties):
        # y_i = sum of thickness of layers below i + h_i/2
        # Note: layers list is from top to bottom, so "below" means higher index
        thickness_below = sum(layer_properties[j]['thickness'] for j in range(i+1, len(layer_properties)))
        layer_prop['y_i'] = thickness_below + layer_prop['thickness']/2
    
    # Step 8: Locate Centroid of Transformed Slab
    numerator = sum(layer_prop['A_i_prime'] * layer_prop['y_i'] for layer_prop in layer_properties)
    denominator = sum(layer_prop['A_i_prime'] for layer_prop in layer_properties)
    
    if denominator == 0:
        return np.nan
        
    Y_bar = numerator / denominator  # centroid of transformed slab
    
    # Step 9: Calculate Moment of Inertia (I) of the Transformed Slab
    I_bar = 0
    for layer_prop in layer_properties:
        # Moment of inertia of layer i about its own centroid
        I_i = (1/12) * layer_prop['b_i_prime'] * (layer_prop['thickness']**3)
        
        # Distance from centroid of layer i to centroid of transformed slab
        d_i = abs(layer_prop['y_i'] - Y_bar)
        
        # Parallel axis theorem: I = I_local + A * d^2
        I_bar += I_i + layer_prop['A_i_prime'] * (d_i**2)
    
    # Step 10: Calculate Equivalent Bending Stiffness (D11) of the Transformed Slab
    D11 = E_ref * I_bar  # N*m^2 (bending stiffness of transformed slab)
    
    return D11


In [17]:
def filter_pits_with_hardness_and_grain_form(pits):
    """
    Filter pits to only include those with complete hardness and grain form data.
    
    Parameters:
    pits: List of pit objects
    
    Returns:
    list: Filtered list of pits with complete data
    """
    filtered_pits = []
    
    for pit in pits:
        all_layers_info = True
        layers = pit.snow_profile.layers
        for layer in layers:
            if layer.hardness is None or layer.grain_form_primary is None:
                all_layers_info = False
                break
        if all_layers_info:
            filtered_pits.append(pit)
            
    return filtered_pits

def filter_pits_with_layer_of_concern(pits):
    """
    Filter pits to only include those with at least one layer of concern.
    
    Parameters:
    pits: List of pit objects
    
    Returns:
    list: Filtered list of pits with layer of concern
    """
    filtered_pits = []
    
    for pit in pits:
        layers = pit.snow_profile.layers
        has_loc = any(layer.layer_of_concern for layer in layers)
        if has_loc:
            filtered_pits.append(pit)
            
    return filtered_pits

def filter_pits_with_ECTP_propagation(pits):
    """
    Filter pits with ECTP results
    
    Parameters:
    pits: List of pit objects

    Returns:
    list: Filtered list of pits with ECTP results
    """
    filtered_pits = []
    
    for pit in pits:
        ECTs_list = pit.stability_tests.ECT  # list of ECTs for pit
        for ECT in ECTs_list:
            if ECT.propagation:
                filtered_pits.append(pit)
                break
    return filtered_pits

def filter_pits_by_location(pits, location_type):
    """
    Filter pits by avalanche location type (crown or flank).
    
    Parameters:
    pits: List of pit objects
    location_type: String, either "crown" or "flank"
    
    Returns:
    list: Filtered list of pits at specified location
    """
    filtered_pits = []
    
    for pit in pits:
        if pit.core_info.location.pit_near_avalanche_location == location_type:
            filtered_pits.append(pit)
            
    return filtered_pits


In [14]:
def find_layer_of_concern(pit):
    """
    Find the layer of concern in a pit.
    
    Parameters:
    pit: A single pit object
    
    Returns:
    layer: The layer of concern, or None if not found
    """
    layers = pit.snow_profile.layers
    
    # Find the layer of concern
    for layer in layers:
        if layer.layer_of_concern:
            return layer
            
    return None

def find_ECTP_propagation_layer(pit):
    """
    Find the ECTP propagation layer in a pit.
    
    Parameters:
    pit: A single pit object
    
    Returns:
    layer: The ECTP propagation layer, or None if not found
    """
    ECTs_list = pit.stability_tests.ECT  # list of ECTs for pit
    layers = pit.snow_profile.layers  # List of layers in pit
    
    for ECT in ECTs_list:  # For each ECT in pit
        if ECT.propagation:  # If ECTP
            prop_layer_depth_top = ECT.depth_top  # depth of top of propagation layer 
            
            # Find the layer of propagation in layers
            for layer in layers:
                if layer.depth_top == prop_layer_depth_top:
                    return layer
                    
    return None


## Find Pits with Hand Hardness and Grain Form



In [ ]:
# Find pits with hardness and grain form info


print("Num pits with hardness and grain form info: ", len(pits_with_hardness_and_grain_form))


Num pits with hardness and grain form info:  12466


In [ ]:
## Set up datasets

# Total
print("Total Pits: ", len(all_pits))

# Pits with hand hardness and grain form
pits_with_hardness_and_grain_form = filter_pits_with_hardness_and_grain_form(all_pits)
print("Pits with hand hardness and grain form: ", len(pits_with_hardness_and_grain_form))

# Pits with indicated layer of concern
pits_with_layer_of_concern = filter_pits_with_layer_of_concern(pits_with_hardness_and_grain_form)
print("Pits with hand hardness and grain form, and layer of concern: ", len(pits_with_layer_of_concern))

# Pits with ECTP propagation layer
pits_with_ECTP_propagation_layer = filter_pits_with_ECTP_propagation(pits_with_layer_of_concern)
print("Pits with hand hardness and grain form, and ECTP propagation layer: ", len(pits_with_ECTP_propagation_layer))

# Pits on Crowns with layer of concern
pits_on_crowns_with_layer_of_concern = filter_pits_by_location(pits_with_layer_of_concern, "crown")
print("Pits on Crowns with layer of concern: ", len(pits_on_crowns_with_layer_of_concern))

# Pits on Flanks with layer of concern
pits_on_flanks_with_layer_of_concern = filter_pits_by_location(pits_with_layer_of_concern, "flank")
print("Pits on Flanks with layer of concern: ", len(pits_on_flanks_with_layer_of_concern))

# NOTE: Totals do not account for pits with layers that do not work with Geldsetzer table


Total Pits:  31170
Pits with hand hardness and grain form:  12466
Pits with hand hardness and grain form, and layer of concern:  9316
Pits with hand hardness and grain form, and ECTP propagation layer:  2692
Pits on Crowns with layer of concern:  172
Pits on Flanks with layer of concern:  80
